In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, BatchNormalization, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

### Data retrieving

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### Data Preprocessing

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,  # normalization of images
    rotation_range = 10, # augmention of images to avoid overfitting
    zoom_range = 0.1,
    width_shift_range = 0.1, 
    height_shift_range = 0.1,
    fill_mode = 'nearest'
)

val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=64, shuffle=True, seed = 42)

val_generator = val_datagen.flow(X_test, y_test, batch_size=64, shuffle=False, seed = 42)

### Model building

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape = (28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same' ),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same' ),
    Conv2D(128, (3, 3), activation='relu', padding='same' ),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Flatten(),

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(10, activation = 'softmax')
])

model.summary()

### Model compiling

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

### Callbacks

In [ ]:
def callbacks(name): 
    return [ 
        EarlyStopping(monitor = 'val_loss', patience = 6), 
        ModelCheckpoint(f'models/{name}', save_best_only=True) # saving the best model
    ]

### Model training

In [ ]:
BATCH_SIZE = 128
EPOCHS = 50

In [ ]:
history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs=EPOCHS,
    callbacks=callbacks('cnn_v1')
)

In [ ]:
## best weights - /models/cnn_v1